### 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense

### 코드 분석

#### 전체 코드

In [ ]:
### 데이터 준비


# 이 부분은 실제 데이터셋에 맞게 조정해야 함 추후에 알고리즘 내부에 다시 한번 재정의
p = np.random.rand(100, 3)  # 가격 데이터
v = np.random.rand(100, 3)  # 거래량 데이터
s = np.random.rand(100, 1)  # 센티먼트 데이터

for t in range(len(p)):
    data = np.concatenate([p[t], v[t], s[t]]) # (p_t,v_t;s_t) 꼴의 데이터셋


### LSTM 모델 불러오기

input_data = np.reshape(data, (1, 1, -1))
input_size = len(p[0]) + len(v[0]) + len(s[0])
lstm_units = 50 # 초기 셀의 개수 설정

def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (예: 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = initialize_ecm_lstm(input_size, lstm_units)


### 초기값 설정

centroids = []
centroids.append(data[0])
radiuses = []
radiuses.append(0)


### ECM-LSTM 알고리즘

def initial_step(data, centroids, radiuses, model):
    predicted_returns = []

    for t in range(len(data)):
        new_data = np.concatenate([p[t], v[t], s[t]])
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

        # 가장 가까운 클러스터 찾기
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

        # 클러스터 업데이트
        if closest_idx is not None and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        else:
            # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(centroids, radiuses):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)
                
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        # LSTM 모델을 사용한 예측
        predicted_return = model.predict(np.reshape(new_data, (1, 1, -1)))
        predicted_returns.append(predicted_return)

    return centroids, radiuses, predicted_returns

### ${p_t} {v_t} {s_t}$ 데이터셋의 형태(행렬구조)의 예시는 아래와 같습니다. -> **티커 별로 가격(종가)데이터와 거래량 데이터 수집할 필요가 있습니다.**

In [ ]:
# 데이터 준비
# 이 부분은 실제 데이터셋에 맞게 조정해야 함
p = np.random.rand(100, 3)  # 가격 데이터
v = np.random.rand(100, 3)  # 거래량 데이터
s = np.random.rand(100, 1)  # 센티먼트 데이터

### 데이터 병합 (${p_t}, {v_t}, {s_t}$)

In [ ]:
for t in range(len(p)):
    data = np.concatenate([p[t], v[t], s[t]]) # (p_t,v_t;s_t) 꼴의 데이터셋

### LSTM 모델 생성

In [ ]:
input_data = np.reshape(data, (1, 1, -1))
input_size = len(p[0]) + len(v[0]) + len(s[0])
listm_units = 50 # 초기 셀의 개수 설정

def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (=Q_t : 예상 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = initialize_ecm_lstm(input_size, lstm_units)

### 입력 데이터 p,v,s를 LSTM에 추가(add)하는 것으로 알고리즘 1의 14 ~ 20(입력층, 출력층, 망각층; 활성함수 : 시그모이드) 까지 자동으로 구현이 됩니다.
### 이 때, 알고리즘의 소문자 $c_t$는 centroid가 아닌 LSTM Cell입니다.

In [ ]:
input_data = np.reshape(data, (1, 1, -1))
listm_units = 50 # 초기 셀의 개수 설정
def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (예: 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### 클러스터링(ECM 부분) 구현. Clustering 패키지를 임포트할 필요가 없이 단순 구현으로 가능한 로직이라고 합니다.

#### 학습 과정이 없는 이유
##### 논문의 "Algorithm 1 ECM-LSTM training and forecasting procedure"에서 model.fit() 부분이 명시적으로 언급되어 있지 않다면, 이는 모델이 지도학습 방식으로 사전 훈련을 거치지 않는다는 것을 의미할 수 있습니다. 이 경우, ECM-LSTM 접근 방식이 일종의 zero-shot learning이나 더 정확히는 온라인 학습(online learning) 방식으로 작동하는 것으로 해석될 수 있습니다.

##### **Zero-shot Learning**은 모델이 학습 과정에서 본 적 없는 새로운 데이터나 클래스에 대해 예측을 수행하는 능력을 의미합니다. 이 경우, 모델은 일반적으로 사전 훈련된 지식(예: 언어 모델, 전이 학습 등)을 바탕으로 새로운 데이터에 대해 예측합니다.

##### **온라인 학습(Online Learning)** 은 데이터가 순차적으로 모델에 제공되고, 모델이 새로운 데이터 포인트마다 즉시 학습을 수행하는 방식입니다. 이 경우, model.fit()은 각각의 새로운 데이터 포인트에 대해 개별적으로 호출되며, 모델은 계속해서 업데이트되고 발전합니다.

#### **ECM-LSTM 알고리즘 구현 :initial_step 함수**

In [ ]:

def initial_step(data, centroids, radiuses, model):
    predicted_returns = []

    for t in range(len(data)):
        new_data = np.concatenate([p[t], v[t], s[t]])
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

        # 가장 가까운 클러스터 찾기
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

        # 클러스터 업데이트
        if closest_idx is not None and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        else:
            # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(centroids, radiuses):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)
                
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        # LSTM 모델을 사용한 예측
        predicted_return = model.predict(np.reshape(new_data, (1, 1, -1)))
        predicted_returns.append(predicted_return)

    return centroids, radiuses, predicted_returns

- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -

In [ ]:
    predicted_returns = []

    for t in range(len(data)):
        new_data = np.concatenate([p[t], v[t], s[t]])
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

#### 모델의 최종 예측 목표인 $Q_t$를 위한 'predicted_returns' 정의
#### 'new_data' = 데이터 병합 (${p_t}, {v_t}, {s_t}$)
#### 'closest_idx' = ECM의 클러스터 중심(Centroid)를 특정 및 갱신 하기 위한 인덱스(클러스터와 가장 가까운)
#### 'min_distance', 's_min_init' $D_{min}$ 과 $(S_{min},i)$를 구하기 위한 도구

- - -- - -- - -- - -- - -

In [ ]:
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

#### $D_{min}$ = $min(||(p_{t,k},v_{t,k},s_t) - C_i||)$ 을 구하는 과정 : 알고리즘 중에 식별됐던 모든 Centroid = $C_t$를 돌면서 최소거리 $D_{min}$구하기
#### || x - y || = x,y 사이의 유클리디언 거리 = np.linalg.norm

- - -

In [ ]:
        if closest_idx is not None and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)

#### if $R_i >= D_{min}$ then
#### Add $(p_{t,k},v_{t,k},s_t)$ to $C_i$  where $D_{min}$ holds; 부분 (알고리즘 8 ~ 10 행)

- - -

In [ ]:
        else:
            # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(centroids, radiuses):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

#### $(S_{min},i)$ = $min(||(p_{t,k},v_{t,k},s_t) - C_i|| + R_i)$ 구하는 과정

- - -

In [ ]:
            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)

#### if $S_{min}$ > $2R_i$ then 부분 (알고리즘 13행) 

- - -

In [ ]:
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)

#### Add $(p_{t,k},v_{t,k},s_t)$ to $C_i$  where $S_{min}$ holds; 부분 (알고리즘 22 ~ 23 행)

- - -

### 각 if 문과 else 문에서 Centroid와 Radius를 업데이트하고 빠져나오게 되면 LSTM을 통해 $\hat{Q}_t$를 예측합니다.